# TSMixer moddeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.model_selection import TimeSeriesSplit
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
# from lightgbm import LGBMClassifier



import joblib
import json
import os
from datetime import datetime

from TSMixer_Classifier_modeling import select_feature_columns
from TSMixer_Classifier_modeling import walk_forward_train_TSMixer_model
from TSMixer_Classifier_modeling import train_model_TSMixer

⚙️ Using device: mps


In [2]:
model_name = "TSMixer_tbm10d" # ИМЯ МОДЕЛИ

In [3]:

# ==============================================================================
# ОСНОВНОЙ СКРИПТ МОДЕЛИРОВАНИЯ
# ==============================================================================

# --- ШАГ 1: Подготовка данных ---


data_folder = "../data/"
final_dataset_filename = 'moex_final_dataset.csv'
TARGET_COLUMN = 'tbm_10d' # Наша цель для предсказания!

# Загружаем финальный датасет
print(f"Загрузка данных из: {data_folder + final_dataset_filename}")
try:
    df = pd.read_csv(data_folder + final_dataset_filename)
    # if features_to_remove:
    #     df = df.drop(columns=features_to_remove)
    #     print(f"Удалено {len(features_to_remove)} неважных признаков.")


    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    print("Данные успешно загружены.")
except FileNotFoundError:
    print("ОШИБКА: Финальный датасет не найден.")
    raise


print(f"\nИсходный размер датасета: {len(df)} строк.")
df.dropna(subset=[TARGET_COLUMN], inplace=True)
df.reset_index(drop=True, inplace=True)
print(f"Размер датасета после фильтрации по CUSUM-событиям: {len(df)} строк.")


# Определяем признаки (X) и цель (y)
# Исключаем все не-признаки: идентификаторы, дату и ВСЕ колонки с метками
# tbm_cols = [col for col in df.columns if col.startswith('tbm_')]
# cols_to_drop = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume'] + tbm_cols

# feature_cols = [col for col in df.columns if col not in cols_to_drop]
feature_cols = select_feature_columns(df)

X = df[feature_cols]
y = df[TARGET_COLUMN].astype(int)





print(f"\nЦелевая переменная (y): {TARGET_COLUMN}")
print(f"Количество признаков (X): {len(feature_cols)}")
print("Признаки:", feature_cols)

Загрузка данных из: ../data/moex_final_dataset.csv
Данные успешно загружены.

Исходный размер датасета: 15493 строк.
Размер датасета после фильтрации по CUSUM-событиям: 15493 строк.

Целевая переменная (y): tbm_10d
Количество признаков (X): 250
Признаки: ['Open', 'High', 'Low', 'Close', 'Volume', 'sarimax_pred_1d_to_today', 'sarimax_pi_width_rel_1d', 'sarimax_pred_3d_to_today', 'sarimax_pi_width_rel_3d', 'sarimax_pred_5d_to_today', 'sarimax_pi_width_rel_5d', 'sarimax_failed', 'sma_3', 'sma_5', 'sma_7', 'sma_10', 'sma_15', 'sma_20', 'sma_30', 'sma_40', 'sma_50', 'sma_70', 'sma_100', 'sma_150', 'sma_200', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'ADX_14', 'ADXR_14_2', 'DMP_14', 'DMN_14', 'rsi_5', 'rsi_7', 'rsi_14', 'rsi_21', 'rsi_30', 'rsi_50', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'STOCHh_14_3_3', 'willr_5', 'willr_7', 'willr_14', 'willr_21', 'willr_30', 'atr_5', 'atr_7', 'atr_14', 'atr_21', 'BBL_20_2.0_2.0', 'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', '

In [4]:
walk_forward_train_TSMixer_model(df, X, y, lookback=32)




=== Split 1/5 ===
Epoch 1/10 | Loss: 2475539.6677
Epoch 2/10 | Loss: 3859.0941
Epoch 3/10 | Loss: 3315.6496
Epoch 4/10 | Loss: 2336.8737
Epoch 5/10 | Loss: 1935.3860
Epoch 6/10 | Loss: 1880.2162
Epoch 7/10 | Loss: 3842.1698
Epoch 8/10 | Loss: 2352.4492
Epoch 9/10 | Loss: 2060.8997
Epoch 10/10 | Loss: 2997.6474
Accuracy: 0.4435
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       727
           0       0.00      0.00      0.00       692
           1       0.44      1.00      0.61      1131

    accuracy                           0.44      2550
   macro avg       0.15      0.33      0.20      2550
weighted avg       0.20      0.44      0.27      2550

y_test[lookback:] = 2615    1
2616    1
2617    1
2618    1
2619    1
       ..
5160    0
5161    1
5162   -1
5163   -1
5164    1
Name: tbm_10d, Length: 2550, dtype: int64

=== Split 2/5 ===


/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being se

Epoch 1/10 | Loss: 1723763.8977
Epoch 2/10 | Loss: 3144.7024
Epoch 3/10 | Loss: 2421.2631
Epoch 4/10 | Loss: 2273.4254
Epoch 5/10 | Loss: 3298.6694
Epoch 6/10 | Loss: 2126.5504
Epoch 7/10 | Loss: 2474.7529
Epoch 8/10 | Loss: 2055.4613
Epoch 9/10 | Loss: 2597.9439
Epoch 10/10 | Loss: 2278.3758
Accuracy: 0.3690
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1036
           0       0.00      0.00      0.00       573
           1       0.37      1.00      0.54       941

    accuracy                           0.37      2550
   macro avg       0.12      0.33      0.18      2550
weighted avg       0.14      0.37      0.20      2550

y_test[lookback:] = 5197    1
5198    1
5199    1
5200   -1
5201    1
       ..
7742    0
7743    0
7744    0
7745    1
7746    0
Name: tbm_10d, Length: 2550, dtype: int64

=== Split 3/5 ===


/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being se

Epoch 1/10 | Loss: 1083441.2365
Epoch 2/10 | Loss: 3493.8855
Epoch 3/10 | Loss: 2990.2723
Epoch 4/10 | Loss: 2696.0488
Epoch 5/10 | Loss: 3236.2684
Epoch 6/10 | Loss: 2409.0828
Epoch 7/10 | Loss: 2428.0161
Epoch 8/10 | Loss: 2249.7535
Epoch 9/10 | Loss: 2048.4572
Epoch 10/10 | Loss: 2173.9589
Accuracy: 0.4494
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       546
           0       0.00      0.00      0.00       858
           1       0.45      1.00      0.62      1146

    accuracy                           0.45      2550
   macro avg       0.15      0.33      0.21      2550
weighted avg       0.20      0.45      0.28      2550

y_test[lookback:] = 7779     0
7780     0
7781     0
7782     0
7783     0
        ..
10324    1
10325    0
10326    1
10327    1
10328    0
Name: tbm_10d, Length: 2550, dtype: int64

=== Split 4/5 ===


...

Epoch 1/10 | Loss: 938844.0448
Epoch 2/10 | Loss: 4560.1946
Epoch 3/10 | Loss: 3694.9315
Epoch 4/10 | Loss: 4147.6493
Epoch 5/10 | Loss: 3677.3883
Epoch 6/10 | Loss: 4086.3469
Epoch 7/10 | Loss: 3867.6447
Epoch 8/10 | Loss: 4059.5237
Epoch 9/10 | Loss: 3004.7626
Epoch 10/10 | Loss: 2978.2187
Accuracy: 0.4576
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       883
           0       0.00      0.00      0.00       500
           1       0.46      1.00      0.63      1167

    accuracy                           0.46      2550
   macro avg       0.15      0.33      0.21      2550
weighted avg       0.21      0.46      0.29      2550

y_test[lookback:] = 10361    1
10362    0
10363    0
10364    1
10365   -1
        ..
12906   -1
12907   -1
12908   -1
12909   -1
12910   -1
Name: tbm_10d, Length: 2550, dtype: int64

=== Split 5/5 ===


/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being se

Epoch 1/10 | Loss: 249053.2167
Epoch 2/10 | Loss: 5128.7876
Epoch 3/10 | Loss: 4870.4458
Epoch 4/10 | Loss: 3546.0743
Epoch 5/10 | Loss: 3838.3104
Epoch 6/10 | Loss: 3499.3711
Epoch 7/10 | Loss: 3727.7345
Epoch 8/10 | Loss: 3620.5478
Epoch 9/10 | Loss: 3850.4176
Epoch 10/10 | Loss: 4005.8188
Accuracy: 0.2800
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1159
           0       0.00      0.00      0.00       677
           1       0.28      1.00      0.44       714

    accuracy                           0.28      2550
   macro avg       0.09      0.33      0.15      2550
weighted avg       0.08      0.28      0.12      2550

y_test[lookback:] = 12943    0
12944   -1
12945   -1
12946   -1
12947   -1
        ..
15488    0
15489    0
15490    0
15491    0
15492    0
Name: tbm_10d, Length: 2550, dtype: int64

=== Summary ===
   Split  Accuracy  ROC_AUC
0      1  0.443529      NaN
1      2  0.369020      NaN
2      3  0.449412      N

/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being se

,Split,Accuracy,ROC_AUC
0,1,0.443529,NaN
1,2,0.369020,NaN
2,3,0.449412,NaN
3,4,0.457647,NaN
4,5,0.280000,NaN


In [5]:
# # --- ШАГ 4: Анализ важности признаков ---

print("\n--- Определение важности признаков ---")
print("Переобучаем модель на всех доступных данных для получения наиболее стабильной оценки...")

# # Обучаем финальную модель на всех данных
final_model = train_model_TSMixer(X, y, epochs=10, lookback=32)


# ToDo Пока не понятно как считать важность признаков для нейросетей
# # Создаем DataFrame с важностью признаков
# feature_importance = pd.DataFrame({
#     'feature': feature_cols,
#     'importance': final_model.feature_importances_ # метод - Mean Decrease in Impurity (Gini Importance)
# }).sort_values('importance', ascending=False)

# print("Топ-20 самых важных признаков:")
# print(feature_importance.head(20))

# # Визуализируем важность
# plt.figure(figsize=(12, 8))
# sns.barplot(x='importance', y='feature', data=feature_importance.head(20), palette='viridis')
# plt.title('Топ-20 самых важных признаков для модели', fontsize=16)
# plt.xlabel('Важность', fontsize=12)
# plt.ylabel('Признак', fontsize=12)
# plt.tight_layout()
# plt.show()


# print("Топ-20 самых НЕ важных признаков:")
# print(feature_importance.tail(20))
# # Визуализируем важность
# plt.figure(figsize=(12, 8))
# sns.barplot(x='importance', y='feature', data=feature_importance.tail(20), palette='viridis')
# plt.title('Топ-20 самых НЕ важных признаков для модели', fontsize=16)
# plt.xlabel('Важность', fontsize=12)
# plt.ylabel('Признак', fontsize=12)
# plt.tight_layout()
# plt.show()



--- Определение важности признаков ---
Переобучаем модель на всех доступных данных для получения наиболее стабильной оценки...
Epoch 1/10 | Loss: 400405.6998
Epoch 2/10 | Loss: 4189.1670
Epoch 3/10 | Loss: 4132.4267
Epoch 4/10 | Loss: 3839.7785
Epoch 5/10 | Loss: 4233.0366
Epoch 6/10 | Loss: 3513.0743
Epoch 7/10 | Loss: 3620.3651
Epoch 8/10 | Loss: 3414.6114
Epoch 9/10 | Loss: 3349.6426
Epoch 10/10 | Loss: 3194.3628


In [6]:
# --- ШАГ 5: Сохранение артефактов модели ---

print("\n--- Сохранение артефактов модели ---")

output_folder = "../models/"

# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# filename_base = f"{model_name}_{timestamp}"
filename_base = model_name

# Создаем папку, если она не существует
os.makedirs(output_folder, exist_ok=True)
print(f"Артефакты будут сохранены в папку: {output_folder}")

# 1. Сохранение самой модели
model_path = os.path.join(output_folder, f"{filename_base}.joblib")
joblib.dump(final_model, model_path)
print(f"Модель сохранена в: {model_path}")

# 2. Сохранение списка признаков
features_path = os.path.join(output_folder, f"{filename_base}_features.json")
features_list = list(X.columns)
with open(features_path, 'w') as f:
    json.dump(features_list, f, indent=4)
print(f"Список признаков сохранен в: {features_path}")

# 3. Сохранение метаданных и результатов
metadata_path = os.path.join(output_folder, f"{filename_base}_metadata.json")

# Собираем все важные параметры и результаты в один словарь
metadata = {
    "model_name": model_name,
    "model_class": type(final_model).__name__,
    "training_timestamp_utc": datetime.utcnow().isoformat(),
    "target_column": TARGET_COLUMN,
    # "confidence_threshold": CONFIDENCE_THRESHOLD,
    "feature_engineering_file": "feature_engineering.py", # Указываем источник фичей
    "training_data_file": final_dataset_filename,
    "model_parameters": None, # final_model.get_params(),
    "feature_importance": None, # ToDo feature_importance.head(15).to_dict('records'),
    # "cross_validation_summary": results_summary.to_dict('records')
}

with open(metadata_path, 'w', encoding='utf-8') as f:
    # Используем indent=4 для красивого форматирования, ensure_ascii=False для кириллицы
    json.dump(metadata, f, indent=4, ensure_ascii=False)


# 2. Сохранение списка признаков
features_path = os.path.join(output_folder, f"{filename_base}_features.json")
features_list = list(X.columns) 
with open(features_path, 'w') as f:
    json.dump(features_list, f, indent=4)
    
print(f"Список признаков сохранен в: {features_path}")
print(f"Метаданные и результаты сохранены в: {metadata_path}")

print("\nВсе артефакты модели успешно сохранены!")


--- Сохранение артефактов модели ---
Артефакты будут сохранены в папку: ../models/
Модель сохранена в: ../models/TSMixer_tbm10d.joblib
Список признаков сохранен в: ../models/TSMixer_tbm10d_features.json
Список признаков сохранен в: ../models/TSMixer_tbm10d_features.json
Метаданные и результаты сохранены в: ../models/TSMixer_tbm10d_metadata.json

Все артефакты модели успешно сохранены!


/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_23170/1835760050.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "training_timestamp_utc": datetime.utcnow().isoformat(),
